<a href="https://colab.research.google.com/github/malraharsh/temp2/blob/master/Skin_Classifier_Image_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os

import pandas as pd
import shutil
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import os
from sklearn.model_selection import train_test_split
import shutil
import tensorflow as tf
import numpy as np
import keras
from keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import scipy.ndimage
from scipy import misc
from glob import glob
from scipy import stats
from sklearn.preprocessing import LabelEncoder, StandardScaler
import skimage
import imageio
import seaborn as sns
from PIL import Image
import glob
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

In [ ]:
%mkdir -p others data

In [ ]:
!cp -n '/content/drive/MyDrive/Internship/Images zip.zip' /content/others/
!cp -n /content/drive/MyDrive/Internship/*.csv /content/data/

In [ ]:
%%capture
!unzip -n '/content/others/Images zip.zip'

In [ ]:
path_data = '/content/Images' #os.getcwd() + "/HAM10000"
# path_metadata = '/content/HAM10000_metadata.csv'

In [ ]:
to_use = ['Herpes Simplex', 'Acne', 'Folliculitis', 'Alopecia', 'Drug Eruption']
# to_use = ['Folliculitis', 'Alopecia', 'Drug Eruption']

In [ ]:
labels = to_use
labels

['Herpes Simplex', 'Acne', 'Folliculitis', 'Alopecia', 'Drug Eruption']

In [ ]:
path_root = '/content/'
path_dest = '/content/destination'
path_train = path_root + 'train'
path_test = path_root + 'test'

# Image

In [ ]:
# delete extra
for folder in os.listdir(path_data):
    if folder in to_use: continue
    folder = os.path.join(path_data, folder)
    print(folder)
    !rm -r "$folder"

In [ ]:
counts = []
for l in to_use:
    cnt = len(os.listdir(os.path.join(path_data, l)))
    counts.extend([l] * cnt)
    print(f'{l} - {cnt}')

Herpes Simplex - 653
Acne - 436
Folliculitis - 128
Alopecia - 104
Drug Eruption - 210


In [ ]:
datagen_train = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        # vertical_flip=True,
        # brightness_range=(0.9,1.1),
        fill_mode='nearest',
        preprocessing_function= keras.applications.mobilenet.preprocess_input)

datagen_test = ImageDataGenerator(preprocessing_function= keras.applications.mobilenet.preprocess_input)

In [ ]:
num_train_samples = len(counts)
num_test_samples = len(counts)
train_batch_size = 32
test_batch_size = 32
image_shape = (224, 224)

In [ ]:
train_steps = np.ceil(num_train_samples / train_batch_size)
test_steps = np.ceil(num_test_samples / test_batch_size)

In [ ]:
batches_train = datagen_train.flow_from_directory(
                                directory=path_data,
                                # subset="training",
                                batch_size=train_batch_size,
                                seed=42,
                                shuffle=True,
                                class_mode="categorical",
                                target_size=image_shape)

batches_test = datagen_test.flow_from_directory(
                                directory=path_data,
                                # subset="training",
                                batch_size=train_batch_size,
                                seed=42,
                                shuffle=False,
                                class_mode="categorical",
                                target_size=image_shape)

Found 1531 images belonging to 5 classes.
Found 1531 images belonging to 5 classes.


In [ ]:
batches_train.class_indices

{'Acne': 0,
 'Alopecia': 1,
 'Drug Eruption': 2,
 'Folliculitis': 3,
 'Herpes Simplex': 4}

Class Weights

In [ ]:
from sklearn.utils import class_weight

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(batches_train.classes),
                                                 batches_train.classes)
class_weights = dict(enumerate(class_weights))

In [ ]:
class_weights

{0: 0.7022935779816514,
 1: 2.9442307692307694,
 2: 1.458095238095238,
 3: 2.3921875,
 4: 0.4689127105666156}

# Models

In [ ]:
import os
import cv2
import requests
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
# import scikitplot as skplt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=image_shape + (3,))
base_model = tf.keras.applications.mobilenet.MobileNet()

17227776/17225924 [==============================] - 0s 0us/step


In [ ]:
for layer in base_model.layers[:-23]:
     layer.trainable = False

top = tf.keras.Sequential()
top.add(Flatten())
top.add(Dense(512, activation = 'relu'))
top.add(BatchNormalization())
top.add(Dropout(0.5))
top.add(Dense(256, activation = 'relu'))
top.add(BatchNormalization())
top.add(Dropout(0.5))
top.add(Dense(256, activation = 'relu'))
top.add(BatchNormalization())
top.add(Dropout(0.5))
top.add(Dense(128, activation = 'relu'))
top.add(BatchNormalization())
top.add(Dropout(0.25))
top.add(Dense(len(labels), activation = 'softmax'))

# top.compile(optimizer=tf.keras.optimizers.Adam(), 
#               loss=keras.losses.categorical_crossentropy, 
#               metrics=['accuracy'])

# top.summary()

In [ ]:
x = base_model.layers[-6].output
outputs = top(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=outputs)

In [ ]:
# model.summary()

In [ ]:
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])

filepath = "model.h5"

checkpoint = ModelCheckpoint(filepath, monitor='val_top_3_accuracy', verbose=1,
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2,
                              verbose=1, mode='max', min_lr=0.00001)

callbacks = [checkpoint, reduce_lr]


In [ ]:
# !cp /content/drive/MyDrive/Files/dermasync/model_c5.h5 /content/model.h5
# model.load_weights('/content/model.h5')

In [ ]:
# model = tf.keras.models.load_model('model.h5', compile=False)

In [ ]:
class_weights = {
    0: 0.7022935779816514,
    1: 2.9442307692307694,
    2: 3,
    3: 3,
    4: 0.4689127105666156}

In [ ]:
history = model.fit(batches_train,
                    steps_per_epoch=train_steps,
                    class_weight=class_weights,
                    validation_data=batches_test,
                    validation_steps=test_steps,
                    epochs=10,
                    # shuffle=True,#
                    verbose=1,
                    # callbacks=callbacks
                    )

Epoch 1/10
48/48 [==============================] - 20s 419ms/step - loss: 0.5692 - categorical_accuracy: 0.7975 - top_2_accuracy: 0.9438 - top_3_accuracy: 0.9804 - val_loss: 1.1151 - val_categorical_accuracy: 0.6675 - val_top_2_accuracy: 0.8086 - val_top_3_accuracy: 0.9693
Epoch 2/10
48/48 [==============================] - 20s 418ms/step - loss: 0.5244 - categorical_accuracy: 0.8001 - top_2_accuracy: 0.9484 - top_3_accuracy: 0.9824 - val_loss: 1.1122 - val_categorical_accuracy: 0.6799 - val_top_2_accuracy: 0.8687 - val_top_3_accuracy: 0.9419
Epoch 3/10
48/48 [==============================] - 20s 415ms/step - loss: 0.4192 - categorical_accuracy: 0.8432 - top_2_accuracy: 0.9621 - top_3_accuracy: 0.9889 - val_loss: 0.6664 - val_categorical_accuracy: 0.8145 - val_top_2_accuracy: 0.9549 - val_top_3_accuracy: 0.9915
Epoch 4/10
48/48 [==============================] - 20s 418ms/step - loss: 0.3952 - categorical_accuracy: 0.8406 - top_2_accuracy: 0.9595 - top_3_accuracy: 0.9882 - val_loss: 

In [ ]:
model.save('model.h5')

In [ ]:
!cp /content/model.h5 /content/drive/MyDrive/Files/dermasync/model_5class_best.h5

In [ ]:
from sklearn.metrics import precision_score, accuracy_score, confusion_matrix, classification_report

In [ ]:
pred_images = y_pred = model.predict(batches_test, verbose=1)

48/48 [==============================] - 3s 62ms/step


In [ ]:
#add shuffle

In [ ]:
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
label2idx = batches_test.class_indices
idx2label = {v:k for k, v in label2idx.items()}
idx2label

{0: 'Acne',
 1: 'Alopecia',
 2: 'Drug Eruption',
 3: 'Folliculitis',
 4: 'Herpes Simplex'}

In [ ]:
y_true = np.array(batches_test.classes)

In [ ]:
accuracy_score(y_true, y_pred)

0.6094056172436316

In [ ]:
precision_score(y_true, y_pred, average='weighted')

0.7799701810625994

In [ ]:
confusion_matrix(y_true, y_pred)

array([[400,  16,   0,  20,   0],
       [  0, 104,   0,   0,   0],
       [  8,  43, 144,  15,   0],
       [  0,  14,   0, 114,   0],
       [198, 154,  48,  82, 171]])

In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.92      0.77       436
           1       0.31      1.00      0.48       104
           2       0.75      0.69      0.72       210
           3       0.49      0.89      0.64       128
           4       1.00      0.26      0.42       653

    accuracy                           0.61      1531
   macro avg       0.64      0.75      0.60      1531
weighted avg       0.78      0.61      0.58      1531

